In [2]:
!pip install langchain-core
!pip install langchain-community
!pip install langchain-groq
!pip install langchain-experimental

In [3]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
# from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = ""

c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
CSV_PATH = 'data/supermarket_sales.csv'
df = pd.read_csv(CSV_PATH)

In [6]:
connection = sqlite3.connect("sales.db")
df.to_sql(name="sales", con=connection)

ValueError: Table 'sales' already exists.

In [7]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str


def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()


class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")
     

In [8]:

llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)

In [9]:
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("human", human)])

chain = prompt | llm
response = chain.invoke(
    {
        "text": "Which company has better models OpenAI or Anthropic? Respond with just the company name."
    }
)

print(response.content)

Anthropic


In [10]:
db = SQLDatabase.from_uri("sqlite:///sales.db")

In [11]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

In [12]:
list_tables.run()

Using Tool: list_tables


'sales'

In [13]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

In [14]:
print(tables_schema.run("sales"))

Using Tool: tables_schema

CREATE TABLE sales (
	"index" INTEGER, 
	"Invoice ID" TEXT, 
	"Branch" TEXT, 
	"City" TEXT, 
	"Customer type" TEXT, 
	"Gender" TEXT, 
	"Product line" TEXT, 
	"Unit price" REAL, 
	"Quantity" INTEGER, 
	"Tax 5%" REAL, 
	"Total" REAL, 
	"Date" TEXT, 
	"Time" TEXT, 
	"Payment" TEXT, 
	cogs REAL, 
	"gross margin percentage" REAL, 
	"gross income" REAL, 
	"Rating" REAL
)

/*
3 rows from sales table:
index	Invoice ID	Branch	City	Customer type	Gender	Product line	Unit price	Quantity	Tax 5%	Total	Date	Time	Payment	cogs	gross margin percentage	gross income	Rating
0	750-67-8428	A	Yangon	Member	Female	Health and beauty	74.69	7	26.1415	548.9715	1/5/2019	13:08	Ewallet	522.83	4.761904762	26.1415	9.1
1	226-31-3081	C	Naypyitaw	Normal	Female	Electronic accessories	15.28	5	3.82	80.22	3/8/2019	10:29	Cash	76.4	4.761904762	3.82	9.6
2	631-41-3108	A	Yangon	Normal	Male	Home and lifestyle	46.33	7	16.2155	340.5255	3/3/2019	13:23	Credit card	324.31	4.761904762	16.2155	7.4
*/


In [15]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

In [16]:
execute_sql.run("SELECT * FROM sales WHERE Total > 100 LIMIT 5")

Using Tool: execute_sql


"[(0, '750-67-8428', 'A', 'Yangon', 'Member', 'Female', 'Health and beauty', 74.69, 7, 26.1415, 548.9715, '1/5/2019', '13:08', 'Ewallet', 522.83, 4.761904762, 26.1415, 9.1), (2, '631-41-3108', 'A', 'Yangon', 'Normal', 'Male', 'Home and lifestyle', 46.33, 7, 16.2155, 340.5255, '3/3/2019', '13:23', 'Credit card', 324.31, 4.761904762, 16.2155, 7.4), (3, '123-19-1176', 'A', 'Yangon', 'Member', 'Male', 'Health and beauty', 58.22, 8, 23.288, 489.048, '1/27/2019', '20:33', 'Ewallet', 465.76, 4.761904762, 23.288, 8.4), (4, '373-73-7910', 'A', 'Yangon', 'Normal', 'Male', 'Sports and travel', 86.31, 7, 30.2085, 634.3785, '2/8/2019', '10:37', 'Ewallet', 604.17, 4.761904762, 30.2085, 5.3), (5, '699-14-3026', 'C', 'Naypyitaw', 'Normal', 'Male', 'Electronic accessories', 85.39, 7, 29.8865, 627.6165, '3/25/2019', '18:30', 'Ewallet', 597.73, 4.761904762, 29.8865, 4.1)]"

In [17]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

In [18]:
check_sql.run("SELECT * WHERE sales > 10000 LIMIT 5 table = sales")

Using Tool: check_sql


'SELECT * FROM sales WHERE sales > 10000 LIMIT 5'

In [19]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [20]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

In [21]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

In [22]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [23]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [24]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [25]:

crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

In [30]:

inputs = {
    "query": "What is the product line buy per Invoice ID 750-67-8428"
}

result = crew.kickoff(inputs=inputs)

 [2024-09-03 22:02:22][DEBUG]: == Working Agent: Senior Database Developer
 [2024-09-03 22:02:22][INFO]: == Starting Task: Extract data that is required for the query What is the product line buy per Invoice ID 750-67-8428.
 [2024-09-03 22:04:00][DEBUG]: == [Senior Database Developer] Task output: [('Health and beauty',)]


 [2024-09-03 22:04:00][DEBUG]: == Working Agent: Senior Data Analyst
 [2024-09-03 22:04:00][INFO]: == Starting Task: Analyze the data from the database and write an analysis for What is the product line buy per Invoice ID 750-67-8428.
 [2024-09-03 22:04:09][DEBUG]: == [Senior Data Analyst] Task output: Based on the provided data, the analysis for the product line bought per Invoice ID 750-67-8428 is as follows:

The data suggests that the product line bought per Invoice ID 750-67-8428 belongs to the category of 'Health and beauty'. This implies that the customer with this invoice ID has purchased products related to health and beauty, such as skincare, haircare, or 

In [31]:
print(result)

**Executive Summary**

**Product Line Analysis for Invoice ID 750-67-8428**
---------------------------------------------

* **Category:** Health and Beauty
* **Product Line:** Skincare, Haircare, or Makeup products
* **Key Finding:** The customer's purchase is related to Health and Beauty products
* **Limitation:** Analysis is based on a single data point and may not be comprehensive
* **Recommendation:** Further analysis of additional data points, such as specific products purchased or quantity, could provide more insights into the customer's buying behavior
